# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__   # 텐서플로 2.0

'2.15.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
# 고객들에게 수집한 정보를 모은 은행의 데이터 집합
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling

In [11]:
# Feature Scaling은 딥러닝에서 필수. 인공신경망을 만들 때 Feature Scaling을 적용해야 함.
# 딥러닝에서 이 과정은 매우 중요하므로 앞에서 인코딩 작업한 feature까지 모두 스케일링 적용.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [12]:
# 완전 연결망으로 이루어진 딥러닝. 시퀀셜 클래스의 객체가 ann을 나타냄 .
# 시퀀셜 클래스는 원래 케라스 라이브러리의 models 모듈을 취함. 새로운 버전의 텐서플로 2.0이 나오면서 케라스가 텐서플로에 통합됨.
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [13]:
# Dense 클래스는 텐서플로와 파이토치에서 많이 사용. 어떤 ANN에서든 Dense 클래스로 새로운 망에 완전 연결층을 추가함
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
 # add() 메서드로 은닉층이든 드롭아웃층이든 뭐든 원하는 것을 추가 가능. 우리가 추가하고자 하는 건 완전 연결층.
 # layers툴로 원하는 층 추가 가능.
 # Dense클래스의 매개변수 units는 뉴런 수와 정확히 일치. 가지고 싶은 히든 뉴런의 수를 지정 가능.
 # 몇 개의 뉴런이 있어야 할까? -> 경험적 판단은 없음. 실험을 기반으로 함.
 # 다양한 하이퍼파라미터로 실험을 해야 함. 무관하거나 과하지 않은 숫자를 하나 골라야 함.
 # 직관 강의에서 완전 연결 신경망 속 숨겨진 층들의 활성화 함수는 reLU함수여하 한다는 것을 배웠음. 따라서 activation='relu'

### Adding the second hidden layer

In [14]:
# 첫번째 은닉층 추가할 때와 같은 코드 입력하면 됨. add() 메서드로 새로운 층 추가 가능.
# 하이퍼파라미터(초매개변수) 값을 맘대로 바꿔봐도 됨. 더 나은 정확도를 얻을 수도 있음.
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [15]:
# 새로운 층을 추가하고 있고 출력층을 포함해 아무 층이나 add() 메서드로 추가 가능
# 그러므로 최종 출력층 추가에도 add() 메서드를 사용함.
# 물론 출력층이 두 번째 은닉층과 완전 연결 되어야 하기 때문에 Dense 클래스를 사용.
# 출력층은 출력 차원을 포함함. 자료의 출력변수는 1 또는 0이라는 이진변수를 예측하고자 함. 따라서 출력차원은 1차원
# 뉴런은 1개만 필요. 만약 종속변수로 3개의 클래스(A, B, C)를 가지는 분류를 한다면 3차원. 이 클래스들간에 순서는 관계 없으므로 인코딩 하면 A = 100, B = 010, C = 001
# 뉴런이 3개 필요함.
# 시그모이드 활성화 함수는 예상값을 얻는 것에서 끝나지 않고 이분형 결과가 1일 가능성도 보여줌
# 여기서는 해당 고객이 은행 이용을 지속할 지를 예측할 것. 더 나아가 각 고객에 대한 은행 이용 지속 가능성도 알게 됨. 이 모든 것은 시그모이드 활성화 함수로 가능
# 이분형이 아니라면 시그모이드가 아닌 소프트맥스를 사용.
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [16]:
# compile() 메서드 안에 총 3개의 매개변수 입력. optimizer, loss(손실함수), metrics <- ANN평가를 위해 동시에 여러 척도를 선택가능 하므로 복수. 우리는 accuracy만 사용.
# 직관 강의에서 최고의 옵티마이저는 확률적 경사 하강법이라고 배움. 확률적 경사 하강법을 수행할 수 있는 옵티마이저 = ADAM
# 이분화된 결과를 예측해야 한다면 손실함수가 항상 따라옴. binary_crossentropy을 입력(이분형)
# 이분형이 아닌 분류를 한다면 categorical_crossentropy 입력(비이분형)
# 회귀를 위한 ANN은 따로 동영상 링크를 줬음.
# metrics에 여러 개를 입력하려면 대괄호로 짝을 지어 입력(리스트 형태)
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100) # 어떤 머신러닝 모델이건 훈련 메서드는 항상 동일
# ANN 훈련 시 첫번째 배치 사이즈인 매개변수 2개를 더 입력.
# 배치 학습은 ANN 훈련 시 항상 효율적이고 성능이 더 좋기 때문.
# 예측값을 실제 결과와 하나씩 비교하는 대신에 손실을 계산
# 배치 사이즈 매개변수는 실제 결과값과 동일한 수와 비교하여 예측값을 정확히 알려줌.
# 배치 사이즈 값으로는 대개 32가 선택됨.(초매개변수)
# 신경망은 시간이 흐르면서 정확도를 개선하기 위해 일정 에폭 수에 걸쳐 훈련을 받아야 함. 에폭 수를 100으로 설정. 너무 적은 수만 아니면 어떤 수든 가능.
# 신경망은 상관관계를 학습하며 일정량의 에폭을 거쳐 서 최상의 예측을 할 수 있어야 함.

Epoch 1/100
250/250 [==============================] - 4s 8ms/step - loss: 0.5157 - accuracy: 0.7961
Epoch 2/100
250/250 [==============================] - 2s 6ms/step - loss: 0.4553 - accuracy: 0.7997
Epoch 3/100
250/250 [==============================] - 3s 11ms/step - loss: 0.4392 - accuracy: 0.8069
Epoch 4/100
250/250 [==============================] - 2s 7ms/step - loss: 0.4318 - accuracy: 0.8064
Epoch 5/100
250/250 [==============================] - 2s 7ms/step - loss: 0.4273 - accuracy: 0.8100
Epoch 6/100
250/250 [==============================] - 2s 6ms/step - loss: 0.4239 - accuracy: 0.8123
Epoch 7/100
250/250 [==============================] - 2s 10ms/step - loss: 0.4208 - accuracy: 0.8133
Epoch 8/100
250/250 [==============================] - 3s 10ms/step - loss: 0.4168 - accuracy: 0.8175
Epoch 9/100
250/250 [==============================] - 3s 13ms/step - loss: 0.4130 - accuracy: 0.8199
Epoch 10/100
250/250 [==============================] - 3s 11ms/step - loss: 0.4083 - a

In [18]:
# 정확도가 0.86에서 수렴. 약 20번째 에폭에서 수렴.(실행할 때마다 다름)
# 100번의 관측 결과 86번의 예측값이 옳았다는 것.

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

In [19]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
# 1. 대괄호 쌍 안에 2차원 배열로 정보 입력. 2. 카테고리 변수에 대한 문자열 입력 대신 더미변수(가변수)값을 입력해야 함. 3. 스케일링 적용.
# 옵티마이저, 손실함수, matrixs로 ANN을 컴파일할 때, 시그모이드 활성화 함수를 선택했으므로
# 가능성의 형태로 예측값을 얻게 됨. 고객의 이용 유지 여부를 0이나 1이라는 최종 결과로 알려주는 것이 아니라
# 고객의 서비스 중단 가능성을 알려줌.

1/1 [==============================] - 0s 174ms/step
[[0.04222162]]


In [20]:
# 가능성을 예측하고 싶지 않다면  > 0.5를 추가하면 이 가능성이 0.5보다 큰지 물어보기 위해 0.5를 임계값으로 선택 가능. 물론 다른 임계값을 넣어도 됨.
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 38ms/step
[[False]]


**Solution**

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [21]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5) # 이분형 예측 결과인 0 또는 1로 바꾸기 위한 작업.
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1507   88]
 [ 190  215]]


0.861